# Análisis de vuelos

In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [ ]:
!pip install xlsxwriter
!pip install tabulate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 175.3/175.3 kB 5.8 MB/s eta 0:00:00


In [ ]:
url = "https://failbondi.fail/?date="
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"
}
months_max_days = { "01": 31, "02": 28, "03": 31, "04": 30, "05": 31, "06": 30, "07": 31, "08": 31, "09": 30, "10": 31, "11": 30, "12": 31 }
month_days = ["01", "02", "03", "04", "05", "06", "07", "08", "09", "10", "11","12","13","14","15","16","17","18","19","20", "21", "22", "23", "24", "25", "26", "27", "28", "29", "30", "31"]

In [ ]:
def get_html_from_url(url, headers):
    response = requests.get(url, headers=headers)
    if response.status_code == 200:
        # Creamos un objeto BeautifulSoup para poder navegar por el HTML.
        soup = BeautifulSoup(response.content, "html.parser")
        return soup
    return None


In [ ]:
def scraping_vuelos(html, fecha):
    # 1. Extraer encabezados
    headers = [th.text.strip() for th in html.find('thead').find_all('th')]

    # 2. Extraer filas
    rows = []
    table_body = html.find('tbody')
    for tr in table_body.find_all('tr'):
        cells = [td.text.strip() for td in tr.find_all('td')]
        rows.append(cells)

    # 3. Crear DataFrame para manipular fácilmente
    df = pd.DataFrame(rows, columns=headers)
    df['fecha'] = fecha

    return df



In [ ]:
import time
import random

def get_report_by_month(year_month, max_days):
    lista_dfs = []
    for i in range(max_days):
        date = year_month + "-" + month_days[i]
        url_link = url + date
        main_content = get_html_from_url(url_link, headers)
        # Ejecutar la función
        try:
            df_iteracion = scraping_vuelos(main_content, date)
            lista_dfs.append(df_iteracion)
        except Exception as e:
            print(f"Error en fecha {date}: {e}")


    df_month = pd.concat(lista_dfs, ignore_index=True)
    print(f"[{year_month}] - Filas obtenidas: {len(df_month)}")
    time.sleep(random.uniform(2, 5))
    print(df_month.head(5))
    return df_month


In [ ]:
year = "2025"
lista_dfs = []

from datetime import datetime

for month, max_days in months_max_days.items():
    year_month = year + "-" + month
    inicio_peticion = time.time()
    lista_dfs.append(get_report_by_month(year_month, max_days))

    fin_peticion = time.time()
    duracion = fin_peticion - inicio_peticion
    hora_actual = datetime.now().strftime('%H:%M:%S')

    print(f"[{hora_actual}] Finalizado: {year_month} | Tiempo: {duracion:.2f}s")

Error en fecha 2025-01-04: 'NoneType' object has no attribute 'find'
Error en fecha 2025-01-05: 'NoneType' object has no attribute 'find'
Error en fecha 2025-01-06: 'NoneType' object has no attribute 'find'
[2025-01] - Filas obtenidas: 2077
     Vuelo                         Ruta Hora Programada Hora Real  \
0  FO 5912  Aeroparque → Rio de Janeiro           13:05             
1  FO 5237       Bariloche → Aeroparque           07:50     19:22   
2  FO 5236           Ezeiza → Bariloche           05:00     15:16   
3  FO 5027             Córdoba → Ezeiza           20:40  02:18 +1   
4  FO 5056         Aeroparque → Mendoza           16:00     21:27   

  Demora en despegar       fecha  
0          Cancelado  2025-01-01  
1   11hs 32min tarde  2025-01-01  
2   10hs 16min tarde  2025-01-01  
3    5hs 38min tarde  2025-01-01  
4    5hs 27min tarde  2025-01-01  
[14:44:58] Finalizado: 2025-01 | Tiempo: 40.79s
[2025-02] - Filas obtenidas: 1968
     Vuelo                  Ruta Hora Programada Hor

In [ ]:
import os
def exportar_datos(lista_maestra, nombre_archivo, formato='csv'):
    """
    Exporta una lista de listas a CSV o Excel.

    :param lista_maestra: La data a guardar.
    :param nombre_archivo: Nombre del archivo (sin extensión).
    :param formato: 'csv' o 'excel'.
    """
    # Convertimos la lista a un DataFrame de Pandas
    #lista_plana = [registro for sublista in lista_maestra for registro in sublista]
    df_year = pd.concat(lista_maestra, ignore_index=True)
    print(df_year[:10])
    # Usaremos 'vuelo', ya que es la primera columna
    columna_referencia = 'Vuelo'

    # 3. Filtramos: Mantenemos solo las filas donde el valor NO sea igual al nombre del encabezado
    df_limpio = df_year[df_year[columna_referencia] != columna_referencia]
    df = pd.DataFrame(df_limpio)
    ruta = ''

    try:
        if formato.lower() == 'csv':
            ruta = f"{nombre_archivo}.csv"
            df.to_csv(ruta, index=False, encoding='utf-8-sig')
            print(f"✅ Archivo CSV guardado como: {ruta}")

        elif formato.lower() == 'excel':
            ruta = f"{nombre_archivo}.xlsx"
            df.to_excel(ruta, index=False, engine='openpyxl')
            print(f"✅ Archivo Excel guardado como: {ruta}")

        else:
            print("❌ Formato no soportado. Usa 'csv' o 'excel'.")

    except Exception as e:
        print(f"Error al exportar: {e}")
    else:
        return ruta

ruta_archivo = exportar_datos(lista_dfs, "reporte_final_anual", formato='csv')
file_size = os.path.getsize(ruta_archivo)

print(f"\n¡Éxito! El archivo '{ruta_archivo}' ha sido creado. Tamaño: {file_size} bytes")

     Vuelo                         Ruta Hora Programada Hora Real  \
0  FO 5912  Aeroparque → Rio de Janeiro           13:05             
1  FO 5237       Bariloche → Aeroparque           07:50     19:22   
2  FO 5236           Ezeiza → Bariloche           05:00     15:16   
3  FO 5027             Córdoba → Ezeiza           20:40  02:18 +1   
4  FO 5056         Aeroparque → Mendoza           16:00     21:27   
5  FO 5057         Mendoza → Aeroparque           18:25     23:44   
6  FO 5472            Córdoba → Neuquen           16:15     21:01   
7  FO 5272           Ezeiza → Bariloche           12:55     16:13   
8  FO 5016         Aeroparque → Córdoba           15:45     19:01   
9  FO 5061             Mendoza → Ezeiza           21:30  00:26 +1   

  Demora en despegar       fecha  
0          Cancelado  2025-01-01  
1   11hs 32min tarde  2025-01-01  
2   10hs 16min tarde  2025-01-01  
3    5hs 38min tarde  2025-01-01  
4    5hs 27min tarde  2025-01-01  
5    5hs 19min tarde  2025-01-

_____
# Reporte

In [ ]:
def read_csv_file(path):
    df = pd.read_csv(path)
    return df

def imprimir_reporte(titulo, datos):
    ancho = max(len(titulo), 25) # Ajusta el ancho mínimo a 25 o al largo del título
    print("=" * 40)
    print(titulo.upper())
    print("-" * ancho)
    print(datos)
    print("=" * 40)


def basic_info_df(df):
    print("=" * 40)
    print("Report")

    imprimir_reporte("Shape:", f"Columns: {df.shape[1]}\t|\tRows: {df.shape[0]}")
    imprimir_reporte("Info", df.info())

    print_df(df.describe().T)

    imprimir_reporte("Null values count:", df.isnull().sum())

    imprimir_reporte("Unique values:", df.nunique())


def print_df(df_desc):
    from tabulate import tabulate

    # 'headers' son las columnas (count, mean, std, etc.)
    # 'tablefmt' puede ser "grid", "fancy_grid", "pipe" o "psql"
    print(tabulate(df_desc, headers='keys', tablefmt='psql'))

In [ ]:
df = read_csv_file("reporte_final_anual.csv")
basic_info_df(df)

Report
SHAPE:
-------------------------
Columns: 6	|	Rows: 20742
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20742 entries, 0 to 20741
Data columns (total 6 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   Vuelo               20742 non-null  object
 1   Ruta                20742 non-null  object
 2   Hora Programada     20742 non-null  object
 3   Hora Real           19456 non-null  object
 4   Demora en despegar  20742 non-null  object
 5   fecha               20742 non-null  object
dtypes: object(6)
memory usage: 972.4+ KB
INFO
-------------------------
None
+--------------------+---------+----------+----------------------+--------+
|                    |   count |   unique | top                  |   freq |
|--------------------+---------+----------+----------------------+--------|
| Vuelo              |   20742 |      284 | FO 5900              |    361 |
| Ruta               |   20742 |       95 | Aeroparque → Córd

In [ ]:
df.describe().T

,count,unique,top,freq
Vuelo,20742,284,FO 5900,361
Ruta,20742,95,Aeroparque → Córdoba,980
Hora Programada,20742,315,14:30,407
Hora Real,19456,1658,20:10,39
Demora en despegar,20742,806,Cancelado,1286
fecha,20742,365,2025-12-19,82
